In [3]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")

conn = sqlite3.connect(DB_PATH)
pd.options.mode.chained_assignment = None

cursor = conn.cursor()

### Extract information

In [4]:

individuals = pd.read_sql_query("SELECT * FROM individuals_kept", conn)
df_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)
df_works = df_works[df_works['individual_wikidata_id'].isin(list(set(individuals.individual_wikidata_id)))]
df_count_works = df_works.groupby('individual_wikidata_id')['work_wikidata_id'].count().reset_index()
df_count_works = df_count_works.sort_values('work_wikidata_id', ascending=False).reset_index(drop=True)


In [5]:
# Load occupations data and process
df_occupations = pd.read_sql_query("SELECT * FROM individual_occupations", conn)
len(set(df_occupations.occupations_name))

df_occupations = df_occupations[['individual_wikidata_id', 'occupations_name']].drop_duplicates().copy()

df_occupations = df_occupations.groupby('individual_wikidata_id')['occupations_name'].apply(lambda x: " | ".join(x))
df_occupations = df_occupations.reset_index()
df_occupations

,individual_wikidata_id,occupations_name
0,Q1000034,mathematician
1,Q100022441,stage actor
2,Q100028706,composer
3,Q100029799,composer
4,Q100031,poet
...,...,...
220465,Q99981206,painter
220466,Q99982212,architect
220467,Q999904,novelist | translator
220468,Q999920,painter | stage actor


In [6]:

# Define occupation categories
occupation_categories = {
    'painter': 'Painter',
    'poet': 'Writer',
    'composer': 'Musician',
    'historian': 'Writer',
    'theologian': 'Writer',
    'architect': 'Architect',
    'sculptor': 'Sculptor',
    'translator': 'Writer',
    'botanist': 'Writer',
    'philosopher': 'Writer',
    'drawer': 'Painter',
    'mathematician': 'Writer',
    'engraver': 'Sculptor',
    'playwright': 'Performing Artist',
    'singer': 'Musician',
    'musician': 'Musician',
    'chemist': 'Writer',
    'linguist': 'Writer',
    'organist': 'Musician',
    'astronomer': 'Writer',
    'entomologist': 'Writer',
    'editor': 'Writer',
    'philologist': 'Writer',
    'stage actor': 'Performing Artist',
    'archaeologist': 'Writer',
    'naturalist': 'Writer',
    'printmaker': 'Painter',
    'physicist': 'Writer',
    'opera singer': 'Musician',
    'novelist': 'Writer',
    'conductor': 'Musician',
    'classical philologist': 'Writer',
    'cartographer': 'Writer',
    'pianist': 'Musician',
    'economist': 'Writer',
    'copperplate engraver': 'Sculptor',
    'illustrator': 'Painter',
    'geologist': 'Writer',
    'zoologist': 'Writer',
    'scientific illustrator': 'Painter',
    'scientist': 'Writer',
    'geographer': 'Writer',
    'art historian': 'Painter',
    'violinist': 'Musician'
}


# Load occupations data and process
df_occupations_meta = pd.read_sql_query("SELECT * FROM individual_occupations", conn)
df_occupations_meta = df_occupations_meta[['individual_wikidata_id', 'occupations_name']].drop_duplicates().copy()
df_occupations_meta['meta_occupation'] = df_occupations_meta['occupations_name'].apply(lambda x: occupation_categories.get(x, None))
df_occupations_meta = df_occupations_meta.dropna()

df_occupations_meta = df_occupations_meta.groupby('individual_wikidata_id')['meta_occupation'].apply(lambda x: " | ".join(set(x)))
df_occupations_meta = df_occupations_meta.reset_index()
df_occupations_meta

,individual_wikidata_id,meta_occupation
0,Q1000034,Writer
1,Q100022441,Performing Artist
2,Q100028706,Musician
3,Q100029799,Musician
4,Q100031,Writer
...,...,...
173542,Q99981206,Painter
173543,Q99982212,Architect
173544,Q999904,Writer
173545,Q999920,Painter | Performing Artist


In [7]:
df_occupations_final = pd.merge(df_occupations, df_occupations_meta, on = 'individual_wikidata_id', how = 'left')
df_occupations_final

,individual_wikidata_id,occupations_name,meta_occupation
0,Q1000034,mathematician,Writer
1,Q100022441,stage actor,Performing Artist
2,Q100028706,composer,Musician
3,Q100029799,composer,Musician
4,Q100031,poet,Writer
...,...,...,...
220465,Q99981206,painter,Painter
220466,Q99982212,architect,Architect
220467,Q999904,novelist | translator,Writer
220468,Q999920,painter | stage actor,Painter | Performing Artist


In [8]:
df_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_regions = df_regions.groupby('individual_wikidata_id')['region_name'].apply(lambda x: " | ".join(x))
df_regions = df_regions.reset_index()
df_regions

,individual_wikidata_id,region_name
0,Q1000034,German world | Germany | Northwestern Europe |...
1,Q100022441,Northern France | France | Northwestern Europe...
2,Q100028706,Southern Italy | Italy | Southwestern Europe |...
3,Q100029799,Northern Italy | Italy | Southwestern Europe |...
4,Q100031,German world | Germany | Northwestern Europe |...
...,...,...
177633,Q99980669,Central Europe | Eastern Europe
177634,Q99982212,Central Europe | Eastern Europe
177635,Q999904,Southwestern Europe | Spain | Western Europe
177636,Q999920,Austria | German world | Northwestern Europe |...


In [9]:
df_main_information = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_main_information = df_main_information[['individual_wikidata_id', 'individual_name', 'birthyear']]

In [10]:
df_final = pd.merge(df_main_information, df_regions, on = 'individual_wikidata_id')
df_final = pd.merge(df_final, df_occupations_final, on = 'individual_wikidata_id', how='left')
df_final = pd.merge(df_final, df_count_works, on = 'individual_wikidata_id', how='left')
df_final['work_wikidata_id'] = df_final['work_wikidata_id'].fillna(0)
df_final = df_final.rename(columns={'work_wikidata_id':'count_works'})
df_final


,individual_wikidata_id,individual_name,birthyear,region_name,occupations_name,meta_occupation,count_works
0,Q1000034,Joseph Dienger,1818.0,German world | Germany | Northwestern Europe |...,mathematician,Writer,0.0
1,Q100022441,Charles Joseph Vanhove,1739.0,Northern France | France | Northwestern Europe...,stage actor,Performing Artist,0.0
2,Q100028706,Luigi Sangermano,1846.0,Southern Italy | Italy | Southwestern Europe |...,composer,Musician,0.0
3,Q100029799,Eugenio Torriani,1825.0,Northern Italy | Italy | Southwestern Europe |...,composer,Musician,0.0
4,Q100031,Kaspar von Stieler,1632.0,German world | Germany | Northwestern Europe |...,poet,Writer,0.0
...,...,...,...,...,...,...,...
177823,Q99980669,Adrian Junga,1550.0,Central Europe | Eastern Europe,theologian,Writer,0.0
177824,Q99982212,Alois Filcík,1841.0,Central Europe | Eastern Europe,architect,Architect,0.0
177825,Q999904,Narcís Oller,1846.0,Southwestern Europe | Spain | Western Europe,novelist | translator,Writer,16.0
177826,Q999920,Joseph Lange,1751.0,Austria | German world | Northwestern Europe |...,painter | stage actor,Painter | Performing Artist,2.0


In [11]:
df_final.to_csv('db_extract/df_count_works.csv')


In [12]:
df_final.sample(5)

,individual_wikidata_id,individual_name,birthyear,region_name,occupations_name,meta_occupation,count_works
161631,Q86376351,Guillaume Grohé,1808.0,Northern France | France | Northwestern Europe...,artist | visual artist,NaN,0.0
114266,Q4755378,Andreas Berger,1584.0,German world | Germany | Northwestern Europe |...,composer,Musician,0.0
110094,Q443320,Elizabeth Montagu,1718.0,British Islands | Northwestern Europe | United...,essayist,NaN,0.0
4433,Q105521716,Adam Niernberger,1759.0,Austria | German world | Northwestern Europe |...,church musician,NaN,0.0
132554,Q57037460,Caterina Bresciani,1722.0,Northern Italy | Italy | Southwestern Europe |...,stage actor,Performing Artist,0.0


In [13]:
# Load works of individuals and process data

# Load works data
# df_ind_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)

# df_created_works_id = pd.read_sql_query("SELECT * FROM created_work_identifiers", conn)